# connect to drive


In [1]:
import os
os.chdir("/")

from google.colab import drive
try:
    drive.mount('content/gdrive', force_remount=True)
except:
    print('你的 drive 掛載不成功')
    
os.chdir("/content/gdrive/My Drive/")

files = []
for r, d, f in os.walk("/content/gdrive/My Drive/bert-ner/BERT-silkrode-sample-ner/model_train/"):
    for file in f:
        
        files.append(os.path.join(r, file))

for f in files:
    print(f)
    
os.chdir("/content/gdrive/My Drive/bert-ner/BERT-silkrode-sample-ner/model_train/")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at content/gdrive
/content/gdrive/My Drive/bert-ner/BERT-silkrode-sample-ner/model_train/optimization.py
/content/gdrive/My Drive/bert-ner/BERT-silkrode-sample-ner/model_train/tokenization.py
/content/gdrive/My Drive/bert-ner/BERT-silkrode-sample-ner/model_train/BERT_NER.py
/content/gdrive/My Drive/bert-ner/BERT-silkrode-sample-ner/model_train/tf_metrics.py
/content/gdrive/My Drive/bert-ner/BERT-silkrode-sample-ner/model_train/run.sh
/content/gdrive/My Drive/bert-ner/BERT-silkr

In [0]:
!python BERT_NER.py \
    --do_train=True \
	  --do_eval=True \
	  --do_predict=True \
    --data_dir=data/ \
    --bert_config_file=base_model/chinese-wwm_L-12_H-768_A-12/bert_config.json \
    --init_checkpoint=base_model/chinese-wwm_L-12_H-768_A-12/bert_model.ckpt \
    --vocab_file=base_model/chinese-wwm_L-12_H-768_A-12/vocab.txt \
    --output_dir=./output/ \
    --num_labels=11 \
    --tags=\[\"o\",\"B-CHA\",\"I-CHA\",\"B-TIM\",\"I-TIM\",\"B-MON\",\"I-MON\",\"B-PER\",\"I-PER\",\"[CLS]\",\"[SEP]\"]

python3: can't open file 'BERT_NER.py': [Errno 2] No such file or directory


# Main

In [0]:
#! usr/bin/env python3
# -*- coding:utf-8 -*-
"""
Copyright 2018 The Google AI Language Team Authors.
BASED ON Google_BERT.
@Author:zhoukaiyin
Adjust code for chinese ner
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import modeling
import optimization
import tokenization
import tensorflow as tf
from sklearn.metrics import f1_score,precision_score,recall_score
from tensorflow.python.ops import math_ops
import tf_metrics
import pickle
import json
import os
from kashgari.corpus import ChineseDailyNerCorpus





traindata_dir = 'data/'
bert_config_file = 'base_model/chinese-wwm_L-12_H-768_A-12/bert_config.json'
task_name = "kashgari"
output_dir = './output/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
init_checkpoint = 'base_model/chinese-wwm_L-12_H-768_A-12/bert_model.ckpt'
do_lower_case = True
max_seq_length =128
use_tpu = False
do_train = True
do_eval = True
do_predict = True
train_batch_size = 8
eval_batch_size = 8
predict_batch_size = 8
learning_rate = 5e-5
num_train_epochs = 5.0
warmup_proportion = 0.1
save_checkpoints_steps = 1000
iterations_per_loop = 1000
vocab_file = './base_model/chinese-wwm_L-12_H-768_A-12/vocab.txt'
master = None
num_tpu_cores = 8
crf_loss_method=True



class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        """Constructs a InputExample.

        Args:
          guid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text = text
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids,):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        #self.label_mask = label_mask


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_data(cls, input_file):
        """Reads a BIO data."""
        with open(input_file) as f:
            lines = []
            words = []
            labels = []
            for line in f:
                contends = line.strip()
                word = line.strip().split(' ')[0]
                label = line.strip().split(' ')[-1]
                if contends.startswith("-DOCSTART-"):
                    words.append('')
                    continue
                # if len(contends) == 0 and words[-1] == '。':
                if len(contends) == 0:
                    l = ' '.join([label for label in labels if len(label) > 0])
                    w = ' '.join([word for word in words if len(word) > 0])
                    lines.append([l, w])
                    words = []
                    labels = []
                    continue
                words.append(word)
                labels.append(label)
            return lines


class NerProcessor(DataProcessor):
    def get_train_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_train.txt")), "train"
        )

    def get_dev_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_dev_and_test.txt")), "dev"
        )

    def get_test_examples(self,data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_dev_and_test.txt")), "test")


    def get_labels(self):
        # prevent potential bug for chinese text mixed with english text
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "[CLS]","[SEP]"]
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "X","[CLS]","[SEP]"]
        global traindata_dir
        examples = self.get_train_examples(traindata_dir)
        tags = ["[CLS]","[SEP]"]
        for e in examples:
            
            for l in e.label.split(" "):
                if l not in tags:
                  tags.append(l)
        return tags
        #return ["o", "B-CHA", "I-CHA", "B-TIM", "I-TIM", "B-MON", "I-MON", "B-PER", "I-PER","[CLS]","[SEP]"]

    def _create_example(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text = tokenization.convert_to_unicode(line[1])
            label = tokenization.convert_to_unicode(line[0])
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples



class kashgariProcessor(DataProcessor):
    """
    from kashgari.corpus import ChineseDailyNerCorpus

    train_x, train_y = ChineseDailyNerCorpus.load_data('train')
    valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
    test_x, test_y  = ChineseDailyNerCorpus.load_data('test')
    """
    
    def get_train_examples(self,*args):
        train_x, train_y = ChineseDailyNerCorpus.load_data('train')
         
        return self._create_example(self.load_data2set(train_x, train_y), "train")

    def get_dev_examples(self,*args):
        valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
        return self._create_example(self.load_data2set(valid_x, valid_y, max_num = 1000), "dev")

    def get_test_examples(self,*args):
        test_x, test_y  = ChineseDailyNerCorpus.load_data('test')
        return self._create_example(self.load_data2set(test_x, test_y, max_num = 1000), "test")


    def load_data2set(self,set_x, set_y, max_num = 50000):
      
      data_set = []
      idx = 0
      sent_text,sent_slot="",""
      for label,ans in zip(set_x,set_y):
          idx+=1
          if idx > max_num and max_num !=-1:
            break
          sent_text = " ".join(label)
          sent_slot = " ".join(ans)
          data_set.append([sent_text,sent_slot])
          
      return data_set
    def get_labels(self):
        # prevent potential bug for chinese text mixed with english text
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "[CLS]","[SEP]"]
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "X","[CLS]","[SEP]"]
        
        examples = self.get_train_examples()
        tags = ["[CLS]","[SEP]"]
        
        for idx,e in enumerate(examples):
            temp_tags = e.label.split(" ")
            for n_tag in temp_tags:
              if n_tag not in tags:
                tags.append(n_tag)
        return tags

    def _create_example(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text = tokenization.convert_to_unicode(line[0])
            label = tokenization.convert_to_unicode(line[1])
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples
def write_tokens(tokens,mode):
    if mode=="test":
        path = os.path.join(output_dir, "token_"+mode+".txt")
        wf = open(path,'a')
        for token in tokens:
            if token!="**NULL**":
                wf.write(token+'\n')
        wf.close()

def convert_single_example(ex_index, example, label_map, max_seq_length, tokenizer,mode):
    textlist = example.text.split(' ')
    labellist = example.label.split(' ')
    tokens = []
    labels = []
    # print(textlist)
    for i, word in enumerate(textlist):
        token = tokenizer.tokenize(word)
        # print(token)
        tokens.extend(token)
        label_1 = labellist[i]
        # print(label_1)
        for m in range(len(token)):
            if m == 0:
                labels.append(label_1)
            else:
                labels.append("X")
        # print(tokens, labels)
    # tokens = tokenizer.tokenize(example.text)
    if len(tokens) >= max_seq_length - 1:
        tokens = tokens[0:(max_seq_length - 2)]
        labels = labels[0:(max_seq_length - 2)]
    ntokens = []
    segment_ids = []
    label_ids = []
    ntokens.append("[CLS]")
    segment_ids.append(0)
    # append("O") or append("[CLS]") not sure!
    label_ids.append(label_map["[CLS]"])
    for i, token in enumerate(tokens):
        ntokens.append(token)
        segment_ids.append(0)
        label_ids.append(label_map[labels[i]])
    ntokens.append("[SEP]")
    segment_ids.append(0)
    # append("O") or append("[SEP]") not sure!
    label_ids.append(label_map["[SEP]"])
    input_ids = tokenizer.convert_tokens_to_ids(ntokens)
    input_mask = [1] * len(input_ids)
    #label_mask = [1] * len(input_ids)
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
        # we don't concerned about it!
        label_ids.append(0)
        ntokens.append("**NULL**")
        #label_mask.append(0)
    # print(len(input_ids))
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    assert len(label_ids) == max_seq_length
    #assert len(label_mask) == max_seq_length

    if ex_index < 5:
        tf.logging.info("*** Example ***")
        tf.logging.info("guid: %s" % (example.guid))
        tf.logging.info("tokens: %s" % " ".join(
            [tokenization.printable_text(x) for x in tokens]))
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        tf.logging.info("label_ids: %s" % " ".join([str(x) for x in label_ids]))
        #tf.logging.info("label_mask: %s" % " ".join([str(x) for x in label_mask]))

    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=label_ids,
        #label_mask = label_mask
    )
    write_tokens(ntokens,mode)
    return feature


def filed_based_convert_examples_to_features(
        examples, label_list, max_seq_length, tokenizer, output_file,mode=None
):
    label_map = {}
    for (i, label) in enumerate(label_list,1):
        label_map[label] = i
    ###
    with open('./output/label2id.pkl','wb') as w:
        pickle.dump(label_map,w)

    writer = tf.python_io.TFRecordWriter(output_file)
    for (ex_index, example) in enumerate(examples):
        if ex_index % 5000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))
        feature = convert_single_example(ex_index, example, label_map, max_seq_length, tokenizer,mode)
        
        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature(feature.label_ids)
        #features["label_mask"] = create_int_feature(feature.label_mask)
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())


def file_based_input_fn_builder(input_file, seq_length, is_training, drop_remainder):

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([seq_length], tf.int64),
        # "label_ids":tf.VarLenFeature(tf.int64),
        #"label_mask": tf.FixedLenFeature([seq_length], tf.int64),
    }

    def _decode_record(record, name_to_features):
        example = tf.parse_single_example(record, name_to_features)
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t
        return example

    def input_fn(params):
        batch_size = params["batch_size"]
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)
        d = d.apply(tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder
        ))
        return d

    return input_fn


def create_model(bert_config, is_training, input_ids, input_mask,
                 segment_ids, labels, num_labels, use_one_hot_embeddings):
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings
    )

    output_layer = model.get_sequence_output()

    hidden_size = output_layer.shape[-1].value

    output_weight = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02)
    )
    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer()
    )
    with tf.variable_scope("loss"):
        global crf_loss_method
        if crf_loss_method is False:
            if is_training:
                output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
            output_layer = tf.reshape(output_layer, [-1, hidden_size])
            logits = tf.matmul(output_layer, output_weight, transpose_b=True)
            logits = tf.nn.bias_add(logits, output_bias)
            logits = tf.reshape(logits, [-1, max_seq_length, num_labels])
            # mask = tf.cast(input_mask,tf.float32)
            # loss = tf.contrib.seq2seq.sequence_loss(logits,labels,mask)
            # return (loss, logits, predict)
            ##########################################################################
            log_probs = tf.nn.log_softmax(logits, axis=-1)
            one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
            per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
            loss = tf.reduce_sum(per_example_loss)
            probabilities = tf.nn.softmax(logits, axis=-1)
            predict = tf.argmax(probabilities,axis=-1)
            
            ##########################################################################
        else:
            logits = tf.layers.dense(
                inputs=output_layer,
                units=num_labels,
                use_bias=True,
                bias_initializer=tf.zeros_initializer(),
                # kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))
                kernel_initializer=tf.contrib.layers.xavier_initializer()
            )
            mask_length = tf.reduce_sum(input_mask, axis=1)

            log_likelihood, transition = tf.contrib.crf.crf_log_likelihood(
                inputs=logits,
                tag_indices=labels,
                sequence_lengths=mask_length)

            decode_tags, best_score =  tf.contrib.crf.crf_decode(
                potentials=logits,
                transition_params=transition,
                sequence_length=mask_length)

            predict = tf.identity(decode_tags, name='slot_predict')
            per_example_loss = -log_likelihood
            loss = tf.reduce_sum(-log_likelihood)
            
        return (loss, per_example_loss, logits, predict)

         
        
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    def model_fn(features, labels, mode, params):
        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        #label_mask = features["label_mask"]
        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss,  per_example_loss,logits,predicts) = create_model(
            bert_config, is_training, input_ids, input_mask,segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)
        tvars = tf.trainable_variables()
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars,init_checkpoint)
            tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
            if use_tpu:
                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()
                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        tf.logging.info("**** Trainable Variables ****")

        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)
        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:
            
            def metric_fn(per_example_loss, label_ids, logits):
            # def metric_fn(label_ids, logits):
                predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                precision = tf_metrics.precision(label_ids,predictions,num_labels,[i for i in range(1,num_labels-2)],average="macro")
                recall = tf_metrics.recall(label_ids,predictions,num_labels,[i for i in range(1,num_labels-2)],average="macro")
                f = tf_metrics.f1(label_ids,predictions,num_labels,[i for i in range(1,num_labels-2)],average="macro")
                #
                return {
                    "eval_precision":precision,
                    "eval_recall":recall,
                    "eval_f": f,
                    #"eval_loss": loss,
                }
            eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
            # eval_metrics = (metric_fn, [label_ids, logits])
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metrics=eval_metrics,
                scaffold_fn=scaffold_fn)
        else:
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode = mode,predictions= predicts,scaffold_fn=scaffold_fn
            )
        return output_spec
    return model_fn


def main():
    tf.logging.set_verbosity(tf.logging.INFO)
    processors = {
        "ner": NerProcessor,
        "kashgari": kashgariProcessor
    }
    if not do_train and not do_eval and not do_predict:
        raise ValueError("At least one of `do_train` or `do_eval` or `do_predict` must be True.")

    bert_config = modeling.BertConfig.from_json_file(bert_config_file)

    if max_seq_length > bert_config.max_position_embeddings:
        raise ValueError(
            "Cannot use sequence length %d because the BERT model "
            "was only trained up to sequence length %d" %
            (max_seq_length, bert_config.max_position_embeddings))
    global task_name,output_dir
    task_name = task_name.lower()
    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
    processor = processors[task_name]()
    global num_labels
    label_list = processor.get_labels()
    num_labels = len(label_list)+1
    tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)
    tpu_cluster_resolver = None
    if use_tpu and False:
        tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu_name, zone=tpu_zone, project=gcp_project)

    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=master,
        model_dir=output_dir,
        save_checkpoints_steps=save_checkpoints_steps,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=iterations_per_loop,
            num_shards=num_tpu_cores,
            per_host_input_for_training=is_per_host))

    train_examples = None
    num_train_steps = None
    num_warmup_steps = None
    global traindata_dir
    if do_train:
        
        train_examples = processor.get_train_examples(traindata_dir)
        num_train_steps = int(
            len(train_examples) / train_batch_size * num_train_epochs)
        num_warmup_steps = int(num_train_steps * warmup_proportion)
    
    model_fn = model_fn_builder(
        bert_config=bert_config,
        num_labels=num_labels+1,
        init_checkpoint=init_checkpoint,
        learning_rate=learning_rate,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=use_tpu,
        use_one_hot_embeddings=use_tpu)

    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=use_tpu,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=train_batch_size,
        eval_batch_size=eval_batch_size,
        predict_batch_size=predict_batch_size)

    if do_train:
        train_file = os.path.join(output_dir, "train.tf_record")
        filed_based_convert_examples_to_features(
            train_examples, label_list, max_seq_length, tokenizer, train_file)
        tf.logging.info("***** Running training *****")
        tf.logging.info("  Num examples = %d", len(train_examples))
        tf.logging.info("  Batch size = %d", train_batch_size)
        tf.logging.info("  Num steps = %d", num_train_steps)
        train_input_fn = file_based_input_fn_builder(
            input_file=train_file,
            seq_length=max_seq_length,
            is_training=True,
            drop_remainder=True)
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    if do_eval:
        eval_examples = processor.get_dev_examples(traindata_dir)
        eval_file = os.path.join(output_dir, "eval.tf_record")
        filed_based_convert_examples_to_features(
            eval_examples, label_list, max_seq_length, tokenizer, eval_file)

        tf.logging.info("***** Running evaluation *****")
        tf.logging.info("  Num examples = %d", len(eval_examples))
        tf.logging.info("  Batch size = %d", eval_batch_size)
        eval_steps = None
        if use_tpu:
            eval_steps = int(len(eval_examples) / eval_batch_size)
        eval_drop_remainder = True if use_tpu else False
        eval_input_fn = file_based_input_fn_builder(
            input_file=eval_file,
            seq_length=max_seq_length,
            is_training=False,
            drop_remainder=eval_drop_remainder)
        result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
        output_eval_file = os.path.join(output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            tf.logging.info("***** Eval results *****")
            for key in sorted(result.keys()):
                tf.logging.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))
    if do_predict:
        token_path = os.path.join(output_dir, "token_test.txt")
        with open('./output/label2id.pkl','rb') as rf:
            label2id = pickle.load(rf)
            id2label = {value:key for key,value in label2id.items()}
        if os.path.exists(token_path):
            os.remove(token_path)
        predict_examples = processor.get_test_examples(traindata_dir)

        predict_file = os.path.join(output_dir, "predict.tf_record")
        filed_based_convert_examples_to_features(predict_examples, label_list,
                                                max_seq_length, tokenizer,
                                                predict_file,mode="test")
                            
        tf.logging.info("***** Running prediction*****")
        tf.logging.info("  Num examples = %d", len(predict_examples))
        tf.logging.info("  Batch size = %d", predict_batch_size)
        if use_tpu:
            # Warning: According to tpu_estimator.py Prediction on TPU is an
            # experimental feature and hence not supported here
            raise ValueError("Prediction in TPU not supported")
        predict_drop_remainder = True if use_tpu else False
        predict_input_fn = file_based_input_fn_builder(
            input_file=predict_file,
            seq_length=max_seq_length,
            is_training=False,
            drop_remainder=predict_drop_remainder)
        


        

        
        


        result = estimator.predict(input_fn=predict_input_fn)
        output_predict_file = os.path.join(output_dir, "label_test.csv")







        with open(output_predict_file,'w') as writer:
            result_array=[]
            for prediction in result:
                for index,id in enumerate(prediction):
                    if id!=0:
                        result_array.append(id2label[id])
            writer.write("\n".join(result_array))
    print("result_array: ", result_array)
    sample_array_text,sample_array_ans = [],[]
    for example in predict_examples:
      
      temp_text = example.text
      temp_label = example.label
      sample_array_text.append("[CLS]")
      sample_array_ans.append("[CLS]")
      sample_array_text.extend(temp_text.split(" ")[:max_seq_length-2])
      sample_array_ans.extend(temp_label.split(" ")[:max_seq_length-2])
      sample_array_text.append("[SEP]")
      sample_array_ans.append("[SEP]")

 

    # output
    with open(os.path.join(output_dir, "test_samples_results.csv"), 'w') as writer:
        output_array = [" "+"\t"+"預測" + '\t' + "正解"+ '\t' + "內容"]
        for p, s, a in zip(result_array,sample_array_text,sample_array_ans):
            token = "o" if a == p else "x"
            output_array.append(token+ '\t'+p + '\t' + a+ '\t' + s)

        writer.write("\n".join(output_array))
    
if __name__ == "__main__":
    
    main()




Output hidden; open in https://colab.research.google.com to view.

# on GitHub

In [7]:
#! usr/bin/env python3
# -*- coding:utf-8 -*-
"""
Copyright 2018 The Google AI Language Team Authors.
BASED ON Google_BERT.
@Author:zhoukaiyin
Adjust code for chinese ner
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import modeling
import optimization
import tokenization
import tensorflow as tf
from sklearn.metrics import f1_score,precision_score,recall_score
from tensorflow.python.ops import math_ops
import tf_metrics
import pickle
import json
import os
from kashgari.corpus import ChineseDailyNerCorpus





traindata_dir = 'data/'
bert_config_file = 'base_model/chinese-wwm_L-12_H-768_A-12/bert_config.json'
task_name = "kashgari"
output_dir = './output/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
init_checkpoint = 'base_model/chinese-wwm_L-12_H-768_A-12/bert_model.ckpt'
do_lower_case = True
max_seq_length =128
use_tpu = False
do_train = True
do_eval = True
do_predict = True
train_batch_size = 16
eval_batch_size = 16
predict_batch_size = 16
learning_rate = 5e-5
num_train_epochs = 5
warmup_proportion = 0.1
save_checkpoints_steps = 1000
iterations_per_loop = 1000
vocab_file = './base_model/chinese-wwm_L-12_H-768_A-12/vocab.txt'
master = None
num_tpu_cores = 8
crf_loss_method=True



class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        """Constructs a InputExample.
        Args:
          guid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text = text
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids,):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        #self.label_mask = label_mask


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_data(cls, input_file):
        """Reads a BIO data."""
        with open(input_file) as f:
            lines = []
            words = []
            labels = []
            for line in f:
                contends = line.strip()
                word = line.strip().split(' ')[0]
                label = line.strip().split(' ')[-1]
                if contends.startswith("-DOCSTART-"):
                    words.append('')
                    continue
                # if len(contends) == 0 and words[-1] == '。':
                if len(contends) == 0:
                    l = ' '.join([label for label in labels if len(label) > 0])
                    w = ' '.join([word for word in words if len(word) > 0])
                    lines.append([l, w])
                    words = []
                    labels = []
                    continue
                words.append(word)
                labels.append(label)
            return lines

class NerProcessor_from_file(DataProcessor):
    def get_train_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_train.txt")), "train"
        )

    def get_dev_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_dev_and_test.txt")), "dev"
        )

    def get_test_examples(self,data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_dev_and_test.txt")), "test")


    def get_labels(self):
        # prevent potential bug for chinese text mixed with english text
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "[CLS]","[SEP]"]
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "X","[CLS]","[SEP]"]
        global traindata_dir
        examples = self.get_train_examples(traindata_dir)
        tags = ["[CLS]","[SEP]"]
        for e in examples:
            
            for l in e.label.split(" "):
                if l not in tags:
                  tags.append(l)
        return tags
        #return ["o", "B-CHA", "I-CHA", "B-TIM", "I-TIM", "B-MON", "I-MON", "B-PER", "I-PER","[CLS]","[SEP]"]

    def _create_example(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text = tokenization.convert_to_unicode(line[1])
            label = tokenization.convert_to_unicode(line[0])
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples
class kashgariProcessor(DataProcessor):
    """
    from kashgari.corpus import ChineseDailyNerCorpus
    train_x, train_y = ChineseDailyNerCorpus.load_data('train')
    valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
    test_x, test_y  = ChineseDailyNerCorpus.load_data('test')
    """
    
    def get_train_examples(self,*args):
        train_x, train_y = ChineseDailyNerCorpus.load_data('train')
         
        return self._create_example(self.load_data2set(train_x, train_y), "train")

    def get_dev_examples(self,*args):
        valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
        return self._create_example(self.load_data2set(valid_x, valid_y), "dev")

    def get_test_examples(self,*args):
        test_x, test_y  = ChineseDailyNerCorpus.load_data('test')
        return self._create_example(self.load_data2set(test_x, test_y,max_num=1000), "test")


    def load_data2set(self,set_x, set_y, max_num = 50000):
      
      data_set = []
      idx = 0
      sent_text,sent_slot="",""
      for label,ans in zip(set_x,set_y):
          idx+=1
          if idx > max_num and max_num !=-1:
            break
          sent_text = " ".join(label)
          sent_slot = " ".join(ans)
          data_set.append([sent_text,sent_slot])
          
          
          
      print("data_set: ",data_set[:50])
      return data_set
    def get_labels(self):
        # prevent potential bug for chinese text mixed with english text
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "[CLS]","[SEP]"]
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "X","[CLS]","[SEP]"]
        examples = self.get_train_examples()
        tags = ["[CLS]","[SEP]"]
        
        for idx,e in enumerate(examples):
            temp_tags = e.label.split(" ")
            for n_tag in temp_tags:
              if n_tag not in tags:
                tags.append(n_tag)
        return tags

    def _create_example(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text = tokenization.convert_to_unicode(line[0])
            label = tokenization.convert_to_unicode(line[1])
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples
def write_tokens(tokens,mode):
    if mode=="test":
        path = os.path.join(output_dir, "token_"+mode+".txt")
        wf = open(path,'a')
        for token in tokens:
            if token!="**NULL**":
                wf.write(token+'\n')
        wf.close()

def convert_single_example(ex_index, example, label_map, max_seq_length, tokenizer,mode):
    textlist = example.text.split(' ')
    labellist = example.label.split(' ')
    tokens = []
    labels = []
    # print(textlist)
    for i, word in enumerate(textlist):
        token = tokenizer.tokenize(word)
        # print(token)
        tokens.extend(token)
        label_1 = labellist[i]
        # print(label_1)
        for m in range(len(token)):
            if m == 0:
                labels.append(label_1)
            else:
                labels.append("X")
        # print(tokens, labels)
    # tokens = tokenizer.tokenize(example.text)
    if len(tokens) >= max_seq_length - 1:
        tokens = tokens[0:(max_seq_length - 2)]
        labels = labels[0:(max_seq_length - 2)]
    ntokens = []
    segment_ids = []
    label_ids = []
    ntokens.append("[CLS]")
    segment_ids.append(0)
    # append("O") or append("[CLS]") not sure!
    label_ids.append(label_map["[CLS]"])
    for i, token in enumerate(tokens):
        ntokens.append(token)
        segment_ids.append(0)
        label_ids.append(label_map[labels[i]])
    ntokens.append("[SEP]")
    segment_ids.append(0)
    # append("O") or append("[SEP]") not sure!
    label_ids.append(label_map["[SEP]"])
    input_ids = tokenizer.convert_tokens_to_ids(ntokens)
    input_mask = [1] * len(input_ids)
    #label_mask = [1] * len(input_ids)
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
        # we don't concerned about it!
        label_ids.append(0)
        ntokens.append("**NULL**")
        #label_mask.append(0)
    # print(len(input_ids))
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    assert len(label_ids) == max_seq_length
    #assert len(label_mask) == max_seq_length

    if ex_index < 5:
        tf.logging.info("*** Example ***")
        tf.logging.info("guid: %s" % (example.guid))
        tf.logging.info("tokens: %s" % " ".join(
            [tokenization.printable_text(x) for x in tokens]))
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        tf.logging.info("label_ids: %s" % " ".join([str(x) for x in label_ids]))
        #tf.logging.info("label_mask: %s" % " ".join([str(x) for x in label_mask]))

    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=label_ids,
        #label_mask = label_mask
    )
    write_tokens(ntokens,mode)
    return feature


def filed_based_convert_examples_to_features(
        examples, label_list, max_seq_length, tokenizer, output_file,mode=None
):
    label_map = {}
    for (i, label) in enumerate(label_list,1):
        label_map[label] = i
    ###
    with open('./output/label2id.pkl','wb') as w:
        pickle.dump(label_map,w)

    writer = tf.python_io.TFRecordWriter(output_file)
    for (ex_index, example) in enumerate(examples):
        if ex_index % 5000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))
        feature = convert_single_example(ex_index, example, label_map, max_seq_length, tokenizer,mode)
        
        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature(feature.label_ids)
        #features["label_mask"] = create_int_feature(feature.label_mask)
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())


def file_based_input_fn_builder(input_file, seq_length, is_training, drop_remainder):

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([seq_length], tf.int64),
        # "label_ids":tf.VarLenFeature(tf.int64),
        #"label_mask": tf.FixedLenFeature([seq_length], tf.int64),
    }

    def _decode_record(record, name_to_features):
        example = tf.parse_single_example(record, name_to_features)
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t
        return example

    def input_fn(params):
        batch_size = params["batch_size"]
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)
        d = d.apply(tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder
        ))
        return d

    return input_fn


def create_model(bert_config, is_training, input_ids, input_mask,
                 segment_ids, labels, num_labels, use_one_hot_embeddings):
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings
    )

    output_layer = model.get_sequence_output()

    hidden_size = output_layer.shape[-1].value

    output_weight = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02)
    )
    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer()
    )
    with tf.variable_scope("loss"):
        global crf_loss_method
        if crf_loss_method is False:
            if is_training:
                output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
            output_layer = tf.reshape(output_layer, [-1, hidden_size])
            logits = tf.matmul(output_layer, output_weight, transpose_b=True)
            logits = tf.nn.bias_add(logits, output_bias)
            logits = tf.reshape(logits, [-1, max_seq_length, num_labels])
            # mask = tf.cast(input_mask,tf.float32)
            # loss = tf.contrib.seq2seq.sequence_loss(logits,labels,mask)
            # return (loss, logits, predict)
            ##########################################################################
            log_probs = tf.nn.log_softmax(logits, axis=-1)
            one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
            per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
            loss = tf.reduce_sum(per_example_loss)
            probabilities = tf.nn.softmax(logits, axis=-1)
            predict = tf.argmax(probabilities,axis=-1)
            
            ##########################################################################
        else:
            logits = tf.layers.dense(
                inputs=output_layer,
                units=num_labels,
                use_bias=True,
                bias_initializer=tf.zeros_initializer(),
                # kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))
                kernel_initializer=tf.contrib.layers.xavier_initializer()
            )
            mask_length = tf.reduce_sum(input_mask, axis=1)

            log_likelihood, transition = tf.contrib.crf.crf_log_likelihood(
                inputs=logits,
                tag_indices=labels,
                sequence_lengths=mask_length)

            decode_tags, best_score =  tf.contrib.crf.crf_decode(
                potentials=logits,
                transition_params=transition,
                sequence_length=mask_length)

            predict = tf.identity(decode_tags, name='slot_predict')
            per_example_loss = -log_likelihood
            loss = tf.reduce_sum(-log_likelihood)
            
        return (loss, per_example_loss, logits, predict)

         
        
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    def model_fn(features, labels, mode, params):
        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        #label_mask = features["label_mask"]
        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss,  per_example_loss,logits,predicts) = create_model(
            bert_config, is_training, input_ids, input_mask,segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)
        tvars = tf.trainable_variables()
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars,init_checkpoint)
            tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
            if use_tpu:
                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()
                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        tf.logging.info("**** Trainable Variables ****")

        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)
        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:
            
            def metric_fn(per_example_loss, label_ids, logits):
            # def metric_fn(label_ids, logits):
                predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                precision = tf_metrics.precision(label_ids,predictions,num_labels,[i for i in range(1,num_labels-2)],average="macro")
                recall = tf_metrics.recall(label_ids,predictions,num_labels,[i for i in range(1,num_labels-2)],average="macro")
                f = tf_metrics.f1(label_ids,predictions,num_labels,[i for i in range(1,num_labels-2)],average="macro")
                #
                return {
                    "eval_precision":precision,
                    "eval_recall":recall,
                    "eval_f": f,
                    #"eval_loss": loss,
                }
            eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
            # eval_metrics = (metric_fn, [label_ids, logits])
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metrics=eval_metrics,
                scaffold_fn=scaffold_fn)
        else:
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode = mode,predictions= predicts,scaffold_fn=scaffold_fn
            )
        return output_spec
    return model_fn


def main():
    tf.logging.set_verbosity(tf.logging.INFO)
    processors = {
        "ner": NerProcessor_from_file,
        "kashgari": kashgariProcessor
    }
    if not do_train and not do_eval and not do_predict:
        raise ValueError("At least one of `do_train` or `do_eval` or `do_predict` must be True.")

    bert_config = modeling.BertConfig.from_json_file(bert_config_file)

    if max_seq_length > bert_config.max_position_embeddings:
        raise ValueError(
            "Cannot use sequence length %d because the BERT model "
            "was only trained up to sequence length %d" %
            (max_seq_length, bert_config.max_position_embeddings))
    global task_name,output_dir
    task_name = task_name.lower()
    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
    processor = processors[task_name]()
    global num_labels
    label_list = processor.get_labels()
    num_labels = len(label_list)+1
    tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)
    tpu_cluster_resolver = None
    if use_tpu and False:
        tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu_name, zone=tpu_zone, project=gcp_project)

    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=master,
        model_dir=output_dir,
        save_checkpoints_steps=save_checkpoints_steps,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=iterations_per_loop,
            num_shards=num_tpu_cores,
            per_host_input_for_training=is_per_host))

    train_examples = None
    num_train_steps = None
    num_warmup_steps = None
    global traindata_dir
    if do_train:
        
        train_examples = processor.get_train_examples(traindata_dir)
        num_train_steps = int(
            len(train_examples) / train_batch_size * num_train_epochs)
        num_warmup_steps = int(num_train_steps * warmup_proportion)
    
    model_fn = model_fn_builder(
        bert_config=bert_config,
        num_labels=num_labels+1,
        init_checkpoint=init_checkpoint,
        learning_rate=learning_rate,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=use_tpu,
        use_one_hot_embeddings=use_tpu)

    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=use_tpu,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=train_batch_size,
        eval_batch_size=eval_batch_size,
        predict_batch_size=predict_batch_size)

    if do_train:
        train_file = os.path.join(output_dir, "train.tf_record")
        filed_based_convert_examples_to_features(
            train_examples, label_list, max_seq_length, tokenizer, train_file)
        tf.logging.info("***** Running training *****")
        tf.logging.info("  Num examples = %d", len(train_examples))
        tf.logging.info("  Batch size = %d", train_batch_size)
        tf.logging.info("  Num steps = %d", num_train_steps)
        train_input_fn = file_based_input_fn_builder(
            input_file=train_file,
            seq_length=max_seq_length,
            is_training=True,
            drop_remainder=True)
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    if do_eval:
        eval_examples = processor.get_dev_examples(traindata_dir)
        eval_file = os.path.join(output_dir, "eval.tf_record")
        filed_based_convert_examples_to_features(
            eval_examples, label_list, max_seq_length, tokenizer, eval_file)

        tf.logging.info("***** Running evaluation *****")
        tf.logging.info("  Num examples = %d", len(eval_examples))
        tf.logging.info("  Batch size = %d", eval_batch_size)
        eval_steps = None
        if use_tpu:
            eval_steps = int(len(eval_examples) / eval_batch_size)
        eval_drop_remainder = True if use_tpu else False
        eval_input_fn = file_based_input_fn_builder(
            input_file=eval_file,
            seq_length=max_seq_length,
            is_training=False,
            drop_remainder=eval_drop_remainder)
        result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
        output_eval_file = os.path.join(output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            tf.logging.info("***** Eval results *****")
            for key in sorted(result.keys()):
                tf.logging.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))
    if do_predict:
        token_path = os.path.join(output_dir, "token_test.txt")
        with open('./output/label2id.pkl','rb') as rf:
            label2id = pickle.load(rf)
            id2label = {value:key for key,value in label2id.items()}
        if os.path.exists(token_path):
            os.remove(token_path)
        predict_examples = processor.get_test_examples(traindata_dir)

        predict_file = os.path.join(output_dir, "predict.tf_record")
        filed_based_convert_examples_to_features(predict_examples, label_list,
                                                max_seq_length, tokenizer,
                                                predict_file,mode="test")
                            
        tf.logging.info("***** Running prediction*****")
        tf.logging.info("  Num examples = %d", len(predict_examples))
        tf.logging.info("  Batch size = %d", predict_batch_size)
        if use_tpu:
            # Warning: According to tpu_estimator.py Prediction on TPU is an
            # experimental feature and hence not supported here
            raise ValueError("Prediction in TPU not supported")
        predict_drop_remainder = True if use_tpu else False
        predict_input_fn = file_based_input_fn_builder(
            input_file=predict_file,
            seq_length=max_seq_length,
            is_training=False,
            drop_remainder=predict_drop_remainder)
        
        result = estimator.predict(input_fn=predict_input_fn)
        output_predict_file = os.path.join(output_dir, "label_test.csv")

if __name__ == "__main__":
    
    main()

data_set:  [['庞 大 的 奥 运 村 工 地 上 ， 一 边 是 1 5 0 0 名 工 人 在 紧 张 有 序 地 施 工 ， 一 边 是 未 来 奥 运 会 的 工 作 人 员 在 各 交 付 使 用 的 场 馆 、 道 路 、 车 站 等 上 岗 实 习 ， 场 馆 设 施 的 管 理 给 人 高 效 、 严 格 的 印 象 。', 'O O O B-LOC I-LOC I-LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'], ['新 型 消 毒 法 使 水 更 纯 净 本 报 讯 北 京 野 林 科 技 有 限 公 司 近 期 推 出 了 新 型 高 效 二 氧 化 氯 发 生 器 ， 这 种 发 生 器 通 过 电 解 盐 产 生 以 二 氧 化 氯 为 主 ， 伴 有 多 种 强 氧 化 气 体 的 混 合 气 体 ， 在 强 力 杀 菌 的 同 时 ， 不 产 生 氯 代 有 机 物 致 癌 物 质 ， 被 建 设 部 、 中 国 预 防 医 学 科 学 院 等 专 家 誉 为 新 一 代 安 全 型 水 消 毒 系 统 。', 'O O O O O O O O O O O O O B-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-ORG I-ORG I-ORG O B-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG O O O O O O O O O O O O O O O O O'], ['她 说 ： “ 捐 一 所 ‘ 希 望 小 学 ’ 要 好 

INFO:tensorflow:Using config: {'_model_dir': './output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fea6cf681d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_ho

INFO:tensorflow:Using config: {'_model_dir': './output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fea6cf681d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_ho

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


...

INFO:tensorflow:***** Running prediction*****


INFO:tensorflow:  Num examples = 1000


INFO:tensorflow:  Num examples = 1000


INFO:tensorflow:  Batch size = 16


INFO:tensorflow:  Batch size = 16


#kashgari

In [2]:
!pip install kashgari

     |████████████████████████████████| 81kB 2.5MB/s 
     |████████████████████████████████| 17.3MB 8.1MB/s 
     |████████████████████████████████| 645kB 68.6MB/s 
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=724798cc50ed6acd268a1802c4580f36a16104af82ea08fc0053af6fc7cb8951
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.14.0-cp36-none-any.whl size=10525 sha256=73abf058ce571717f74f9721d469567713b599ef1e3cb6060afc15e4f78480c2
  Stored in directory: /root/.cache/pip/wheels/ec/d8/06/ba8216a77a55b8ba4a5c3932c7df93e87eeaea83ced27822aa
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=d6461e6112ee0065ffe18a7efc44d166483ba084e4ee37f67a8855c06f975ec9
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel f

In [0]:

from kashgari.corpus import ChineseDailyNerCorpus

train_x, train_y = ChineseDailyNerCorpus.load_data('train')
valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
test_x, test_y  = ChineseDailyNerCorpus.load_data('test')

def load_data2set(set_x, set_y, max_num = 50000):
      
      data_set = []
      idx = 0
      sent_text,sent_slot="",""
      for label,ans in zip(set_x,set_y):
          idx+=1
          if idx > max_num and max_num !=-1:
            break
          sent_text = " ".join(label)
          sent_slot = " ".join(ans)
          data_set.append([sent_text,sent_slot])
          
          
          
      print("data_set: ",data_set[:50])
      return data_set
print(f"train data count: {len(train_x)}")
print(f"validate data count: {len(valid_x)}")
print(f"test data count: {len(test_x)}")


load_data2set(train_x[:10],train_y[:10])

train data count: 20864
validate data count: 2318
test data count: 4636
data_set:  [['门 诊 病 人 可 先 就 诊 后 挂 号 ； 急 诊 病 人 先 抢 救 后 补 交 费 用 ； 将 原 小 洞 式 的 挂 号 、 划 价 、 收 费 、 取 药 窗 口 扩 大 ， 病 人 与 工 作 人 员 可 以 面 对 面 的 解 答 问 题 ， 贴 近 了 患 者 ， 向 开 放 式 医 疗 服 务 迈 了 一 大 步 ， 受 到 群 众 的 好 评 。', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'], ['近 来 多 次 进 行 反 华 叫 嚣 的 印 度 国 防 部 长 费 尔 南 德 斯 今 天 在 印 度 议 会 上 院 再 次 诬 蔑 中 国 侵 占 印 度 领 土 3 8 0 0 0 平 方 公 里 。', 'O O O O O O O B-LOC O O O B-ORG I-ORG I-ORG I-ORG I-ORG O B-PER I-PER I-PER I-PER I-PER O O O B-ORG I-ORG I-ORG I-ORG I-ORG I-ORG O O O O B-LOC I-LOC O O B-LOC I-LOC O O O O O O O O O O O O'], ['用 户 在 该 取 款 机 上 取 钱 时 ， 一 台 摄 像 机 首 先 要 对 用 户 的 眼 睛 进 行 扫 描 ， 然 后 将 扫 描 图 像 转 化 成 数 字 信 息 与 数 据 库 中 的 资 料 核 对 ， 以 对 用 户 的 身 份 进 行 检 验 。', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O

[['门 诊 病 人 可 先 就 诊 后 挂 号 ； 急 诊 病 人 先 抢 救 后 补 交 费 用 ； 将 原 小 洞 式 的 挂 号 、 划 价 、 收 费 、 取 药 窗 口 扩 大 ， 病 人 与 工 作 人 员 可 以 面 对 面 的 解 答 问 题 ， 贴 近 了 患 者 ， 向 开 放 式 医 疗 服 务 迈 了 一 大 步 ， 受 到 群 众 的 好 评 。',
  'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'],
 ['近 来 多 次 进 行 反 华 叫 嚣 的 印 度 国 防 部 长 费 尔 南 德 斯 今 天 在 印 度 议 会 上 院 再 次 诬 蔑 中 国 侵 占 印 度 领 土 3 8 0 0 0 平 方 公 里 。',
  'O O O O O O O B-LOC O O O B-ORG I-ORG I-ORG I-ORG I-ORG O B-PER I-PER I-PER I-PER I-PER O O O B-ORG I-ORG I-ORG I-ORG I-ORG I-ORG O O O O B-LOC I-LOC O O B-LOC I-LOC O O O O O O O O O O O O'],
 ['用 户 在 该 取 款 机 上 取 钱 时 ， 一 台 摄 像 机 首 先 要 对 用 户 的 眼 睛 进 行 扫 描 ， 然 后 将 扫 描 图 像 转 化 成 数 字 信 息 与 数 据 库 中 的 资 料 核 对 ， 以 对 用 户 的 身 份 进 行 检 验 。',
  'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'],
 ['● 人 民 文 学 ： 《 小 冬 与 车 》 

#Data Processor Test

In [0]:
import tokenization
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        """Constructs a InputExample.

        Args:
          guid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text = text
        self.label = label
        
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_data(cls, input_file):
        """Reads a BIO data."""
        with open(input_file) as f:
            lines = []
            words = []
            labels = []
            for line in f:
                contends = line.strip()
                word = line.strip().split(' ')[0]
                label = line.strip().split(' ')[-1]
                if contends.startswith("-DOCSTART-"):
                    words.append('')
                    continue
                # if len(contends) == 0 and words[-1] == '。':
                if len(contends) == 0:
                    l = ' '.join([label for label in labels if len(label) > 0])
                    w = ' '.join([word for word in words if len(word) > 0])
                    lines.append([l, w])
                    words = []
                    labels = []
                    continue
                words.append(word)
                labels.append(label)
            return lines


class NerProcessor(DataProcessor):
    def get_train_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_train.txt")), "train"
        )

    def get_dev_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_dev_and_test.txt")), "dev"
        )

    def get_test_examples(self,data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "for_dev_and_test.txt")), "test")


    def get_labels(self):
        # prevent potential bug for chinese text mixed with english text
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "[CLS]","[SEP]"]
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "X","[CLS]","[SEP]"]
        traindata_dir = 'data/'
        examples = self.get_train_examples(traindata_dir)
        tags = ["[CLS]","[SEP]"]
        for e in examples:
            
            for l in e.label.split(" "):
                if l not in tags:
                  tags.append(l)
        return tags
        #return ["o", "B-CHA", "I-CHA", "B-TIM", "I-TIM", "B-MON", "I-MON", "B-PER", "I-PER","[CLS]","[SEP]"]

    def _create_example(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text = tokenization.convert_to_unicode(line[1])
            label = tokenization.convert_to_unicode(line[0])
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples

class kashgariProcessor(DataProcessor):
    """
    from kashgari.corpus import ChineseDailyNerCorpus

    train_x, train_y = ChineseDailyNerCorpus.load_data('train')
    valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
    test_x, test_y  = ChineseDailyNerCorpus.load_data('test')
    """
    
    def get_train_examples(self,*args):
        train_x, train_y = ChineseDailyNerCorpus.load_data('train')
         
        return self._create_example(self.load_data2set(train_x, train_y), "train")

    def get_dev_examples(self,*args):
        valid_x, valid_y = ChineseDailyNerCorpus.load_data('validate')
        return self._create_example(self.load_data2set(valid_x, valid_y, max_num = 1000), "dev")

    def get_test_examples(self,*args):
        test_x, test_y  = ChineseDailyNerCorpus.load_data('test')
        return self._create_example(self.load_data2set(test_x, test_y, max_num = 1000), "test")


    def load_data2set(self,set_x, set_y, max_num = 50000):
      
      data_set = []
      idx = 0
      sent_text,sent_slot="",""
      for label,ans in zip(set_x,set_y):
          idx+=1
          if idx > max_num and max_num !=-1:
            break
          sent_text = " ".join(label)
          sent_slot = " ".join(ans)
          data_set.append([sent_text,sent_slot])
          
      return data_set
    def get_labels(self):
        # prevent potential bug for chinese text mixed with english text
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "[CLS]","[SEP]"]
        # return ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "X","[CLS]","[SEP]"]
        
        examples = self.get_train_examples()
        tags = ["[CLS]","[SEP]"]
        
        for idx,e in enumerate(examples):
            temp_tags = e.label.split(" ")
            for n_tag in temp_tags:
              if n_tag not in tags:
                tags.append(n_tag)
        return tags

    def _create_example(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text = tokenization.convert_to_unicode(line[0])
            label = tokenization.convert_to_unicode(line[1])
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples
a = NerProcessor()
b = kashgariProcessor()
#print(a.get_train_examples('./data/'))
print("\t",len(a.get_train_examples('./data/')[0].text.split(" ")),len(a.get_train_examples('./data/')[0].label.split(" ")))
print("\t",[a.get_train_examples('./data/')[0].text,a.get_train_examples('./data/')[0].label])


#print(b.get_train_examples())
train_example = b.get_train_examples()[0]
train_example_text = train_example.text
train_example_label = train_example.label
print("\t",len(train_example_text.split(" ")),len(train_example_label.split(" ")))
print("\t",train_example_text,train_example_label)



	 5 5
	 ['支 付 宝 定 额', 'B-CHA I-CHA I-CHA I-CHA I-CHA']
	 47 47
	 据 统 计 ， 委 内 瑞 拉 的 石 油 储 量 高 达 7 3 5 亿 桶 ， 由 于 其 靠 近 美 国 、 加 拿 大 等 工 业 大 国 ， 石 油 市 场 极 其 广 阔 。 O O O O B-LOC I-LOC I-LOC I-LOC O O O O O O O O O O O O O O O O O O B-LOC I-LOC O B-LOC I-LOC I-LOC O O O O O O O O O O O O O O O


In [0]:
a = NerProcessor()
b = kashgariProcessor()
print(b.get_labels(), a.get_labels())

['[CLS]', '[SEP]', 'X', 'O', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER'] ['[CLS]', '[SEP]', 'B-CHA', 'I-CHA', 'B-TIM', 'I-TIM', 'o', 'B-MON', 'I-MON', 'B-PER', 'I-PER']
